### Streaming Data 처리
- Streaming 데이터를 처리할 때 DataFrame을 사용하기 위해서는
    - Spark Session(즉 spark 변수 사용)을 사용하여 처리
- ssc에서는 DataFrame의 변환이 불가능 즉 .toDF 변화를 실행하면 오류 발생
    - RDD로 계속 작업
- Dataframe의 데이터 입출력 함수를 제공
|     |Read|Write|
|---|---|---|
|일반 데이터  |spark.read|spark.write|
|Stream 데이터|spark.readStream|spark.writeStream|

### 예제
1. 콘솔 입력 데이터의 단어 발생빈도 측정.
2. Dessert menu 주문이 스트리밍 데이터로 입력되면,
   Menu별 가격 데이터를 사용하여 주문별 금액을 계산하여 출력
3. Menu별로 시간대별로 주문 현황이 어떻는가를 집계

### DataFrame Read
- File source
    - Format 사용
        - csv, json 등의 형식으로 데이터를 읽을 수 있음
- Socket source
    - text string으로 읽음.
        - 1개의 컬럼으로 구성된 DataFrame이 만들어짐.
        - 여러 개의 컬럼으로 구성된 Schema를 사용할 수 없음.
    - 테스트용으로만 사용 권장
        - End-to-end fault tolerance를 제공하지 않음.

### SturcturedStreaming Input Source
- socket source
    - 터미널 1:
        - spark.readStream.format("socket").option("host","localhost").option("port",9999).load()
    - 터미널 2:
        - nc -lvp 9999
        
- Files in a directory
    - spark.readStream.schema(mySchema).format("csv").load("/my/path/input")
    
- Apache Kafka

### StructuredStreaming Output Sink
- Console
    - Queryname.writeStream.format("console").option("checkpointLocation","/path/to/directory").outputMode("append").start()
- Memory
    - Queryname.writeStream.queryName("table name").format("memory").outputMode("complete").start()
- File
    - Queryname.writeStream.format("csv").option("path","/path/to/directory").option("checkpointLocation","/path/to/directory").outputMode("append").start()

In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
import pyspark

In [4]:
import pyspark.sql

In [5]:
from pyspark.sql.functions import *

## SparkSession

In [6]:
spark = pyspark.sql.SparkSession  \
.builder \
.master('local[2]') \
.getOrCreate()

In [7]:
linesDF = spark.readStream \
.format("socket") \
.option("host", "localhost") \
.option("port", 9999) \
.load()

In [8]:
linesDF.printSchema()

root
 |-- value: string (nullable = true)



In [9]:
type(linesDF)

pyspark.sql.dataframe.DataFrame

In [10]:
words = linesDF.select(explode(split(linesDF.value," ")).alias("word"))

In [12]:
words.printSchema()

root
 |-- word: string (nullable = true)



In [11]:
wordCount = words.groupby("word").count()

In [13]:
wordCount.printSchema()

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = false)



In [14]:
type(wordCount)

pyspark.sql.dataframe.DataFrame

In [16]:
Query = wordCount.writeStream.format("console")  \
.outputMode("complete")  \
.start()

In [17]:
type(Query)

pyspark.sql.streaming.StreamingQuery

In [ ]:
# Query.awaitTermination()